### Find albums in Spotify and add songs to a playlist

In [1]:
import requests
import json
!pip install pandas
import pandas as pd
!pip install jsontable
import jsontable

     |████████████████████████████████| 9.9 MB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 15.3 MB 60.3 MB/s eta 0:00:01


### Token Management
Right now I have to manually refresh the token via https://developer.spotify.com/console

In [2]:
TOKEN = "BQCr7ON5mIpeT7iC35UDbycICJVabhKXeHR2Qgan7DVke97o042VO0wQQD1nSze86CCxFAt-8j_hYOIejvpb2qKz9ED3L02e4DrWYEe8j5C80s01E3sRx43rvkP75GQVuTehoigmhUf0oqBfZPwHw8i4-uIsv4HrIO2t-XBbAEhx5uqGYNkRYkBQcTxneWPAIaCLvR9p7D_ConMTQOKNpXYt1XyxcGjy"

### Header Construction

In [5]:
header = {"Authorization": "Bearer {}".format(TOKEN)}

### Request Construction

In [6]:
queryURL = "search?q=album:%22bad%20blood%22%20artist:bastille&type=album"
target = "https://api.spotify.com/v1/{}".format(queryURL)
request = requests.get(target, headers = header).json()
print(json.dumps(request, indent=2))

{
  "albums": {
    "href": "https://api.spotify.com/v1/search?query=album%3A%22bad+blood%22+artist%3Abastille&type=album&offset=0&limit=20",
    "items": [
      {
        "album_type": "album",
        "artists": [
          {
            "external_urls": {
              "spotify": "https://open.spotify.com/artist/7EQ0qTo7fWT7DPxmxtSYEc"
            },
            "href": "https://api.spotify.com/v1/artists/7EQ0qTo7fWT7DPxmxtSYEc",
            "id": "7EQ0qTo7fWT7DPxmxtSYEc",
            "name": "Bastille",
            "type": "artist",
            "uri": "spotify:artist:7EQ0qTo7fWT7DPxmxtSYEc"
          }
        ],
        "available_markets": [
          "US"
        ],
        "external_urls": {
          "spotify": "https://open.spotify.com/album/64fQ94AVziavTPdnkCS6Nj"
        },
        "href": "https://api.spotify.com/v1/albums/64fQ94AVziavTPdnkCS6Nj",
        "id": "64fQ94AVziavTPdnkCS6Nj",
        "images": [
          {
            "height": 640,
            "url": "https:/

### Pull Album ID Out of Response
Parse some key fields, and sort by both release date and number of tracks

In [7]:
paths = [{"$.albums.items.id":"ID"}, {"$.albums.items.name":"Album Name"}, {"$.albums.items.total_tracks":"Tracks"}, {"$.albums.items.release_date":"Release Date"}]
converter = jsontable.converter()
converter.set_paths(paths)
resultTable = converter.convert_json(request)
df = pd.DataFrame(data=resultTable)
df.columns = ["Album ID", "Album Name", "Tracks", "Release Date"]
df = df.drop(df.index[0])
df = df.sort_values(["Release Date", "Tracks"], ascending = (False, False))
newest = pd.DataFrame(data=df.iloc[[0]])
newAlbums = pd.DataFrame()
newAlbums = newAlbums.append(newest)
newAlbums.style

,Album ID,Album Name,Tracks,Release Date
2,5G6oMu9zNW2acdV0lqzI3L,All This Bad Blood,25,2013-01-01


### Create Spotify Playlist

In [28]:
userID = "t1e947vjw65dme71utp690f50"
createPlaylistTarget = "https://api.spotify.com/v1/users/{}/playlists".format(userID)
payload = {"name": "Sample Playlist", "description": "Created from Jupyter notebook", "public": False}
payload = json.dumps(payload, indent=2)
createPlaylistPOST = requests.post(createPlaylistTarget, headers = header, data = payload).json()
print(json.dumps(createPlaylistPOST, indent=2))

{
  "collaborative": false,
  "description": "Created from Jupyter notebook",
  "external_urls": {
    "spotify": "https://open.spotify.com/playlist/25NbFuQM1kDleGz1bjGJ2x"
  },
  "followers": {
    "href": null,
    "total": 0
  },
  "href": "https://api.spotify.com/v1/playlists/25NbFuQM1kDleGz1bjGJ2x",
  "id": "25NbFuQM1kDleGz1bjGJ2x",
  "images": [],
  "name": "Sample Playlist",
  "owner": {
    "display_name": "Patrick",
    "external_urls": {
      "spotify": "https://open.spotify.com/user/t1e947vjw65dme71utp690f50"
    },
    "href": "https://api.spotify.com/v1/users/t1e947vjw65dme71utp690f50",
    "id": "t1e947vjw65dme71utp690f50",
    "type": "user",
    "uri": "spotify:user:t1e947vjw65dme71utp690f50"
  },
  "primary_color": null,
  "public": false,
  "snapshot_id": "MSxlMTdkMDA3ZTc2ZTA2MTRkYjI1NWNiYzlmNmY0ZDIwMTQ3MmRjYTUw",
  "tracks": {
    "href": "https://api.spotify.com/v1/playlists/25NbFuQM1kDleGz1bjGJ2x/tracks",
    "items": [],
    "limit": 100,
    "next": null,
    "o

Retrieve User Information

In [19]:
getUserTarget = "https://api.spotify.com/v1/users/{}".format(userID)
getUser = requests.get(getUserTarget, headers = header).json()
print(json.dumps(getUser, indent=2))

{
  "display_name": "Patrick",
  "external_urls": {
    "spotify": "https://open.spotify.com/user/t1e947vjw65dme71utp690f50"
  },
  "followers": {
    "href": null,
    "total": 1
  },
  "href": "https://api.spotify.com/v1/users/t1e947vjw65dme71utp690f50",
  "id": "t1e947vjw65dme71utp690f50",
  "images": [
    {
      "height": null,
      "url": "https://i.scdn.co/image/ab6775700000ee85804e93a20b51ae91923b1a51",
      "width": null
    }
  ],
  "type": "user",
  "uri": "spotify:user:t1e947vjw65dme71utp690f50"
}
